In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import model_selection, linear_model, preprocessing, ensemble, metrics, svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.utils import shuffle
%pylab inline

Populating the interactive namespace from numpy and matplotlib


c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['shuffle', 'plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
train_data = pd.read_csv('train 2.csv', sep =',')

In [4]:
print (train_data.shape)
train_data.head()

(30000, 231)


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,labels
0,NaN,NaN,NaN,NaN,NaN,1176.0,7.0,NaN,NaN,NaN,...,VjJAgOW,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
1,NaN,NaN,NaN,NaN,NaN,1813.0,14.0,NaN,NaN,NaN,...,catzS2D,LM8l689qOp,NaN,ELof,3Cy4,ZI9m,ib5G6X1eUxUn6,mj86,NaN,-1
2,NaN,NaN,NaN,NaN,NaN,777.0,7.0,NaN,NaN,NaN,...,jjOYyXX,LM8l689qOp,NaN,NaN,Qu4f,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
3,NaN,NaN,NaN,NaN,NaN,1554.0,7.0,NaN,NaN,NaN,...,MHP8mHO,LM8l689qOp,NaN,ELof,7P5s,nIGXDli,F2FyR07IdsN7I,mj86,NaN,1
4,NaN,NaN,NaN,NaN,NaN,980.0,0.0,NaN,NaN,NaN,...,C6OGkwT,LM8l689qOp,NaN,NaN,Xa3G,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,-1


In [5]:
from sklearn.utils import shuffle
# так как классы не сбалансированы проводим oversampling, добавляем необходимое количество строк с классом "отток" так, 
# чтобы была пропорция классов примерно 1:1
def oversampling(data):
    dataChurn = data[data['labels'] == 1]
    dataNoChurn = data[data['labels'] == -1]
    alpha = (int)(dataNoChurn.shape[0]/dataChurn.shape[0])
    dataChurnRet = dataChurn 
    for i in range(alpha):
        dataChurnRet = dataChurnRet.append(dataChurn)
    dataNoChurn = dataNoChurn.append(dataChurnRet)
    dataNoChurn = shuffle(dataNoChurn)
    return dataNoChurn.iloc[:,:230], dataNoChurn.iloc[:,230:]
# ради эксперимента попробовал undersampling. Результаты не намного хуже, чем с oversampling'ом
def undersampling(data):
    dataChurn = data[data['labels'] == 1]
    dataNoChurn = data[data['labels'] == -1]
    countFirst = len(dataChurn)
    dataChurn = dataChurn.append(dataNoChurn.iloc[:countFirst,:])
    dataChurn = shuffle(dataChurn)
    return dataChurn.iloc[:,:230], dataChurn.iloc[:,230:]

# 1. Преобразование данных:

In [6]:
#заменяем все пропуски в датасете на нули:
train_data = train_data.fillna(0)
#проводим undersampling для того, чтобы сделать пропорцию классов 1:1 :
x_data, y_data = undersampling(train_data)

In [7]:
le = preprocessing.LabelEncoder()

# Encode first DataFrame 2 (where there is a combination of floats and strings)
x_data = x_data.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
print("x_data encoded")

x_data encoded


# 2. Построение моделей и оценка качества кросс-валидации:

In [8]:
# печатаем метрики по кросс-валидации
def get_cv_roc_pr_rc(clf, skf, X, y):
    roc_auc = []
    pr_auc = []
    precision = []
    recall = []
    models = []
    for train, test in skf.split(X, y):
        models.append(clf.fit(X.iloc[train,:], y.iloc[train]))
        y_pred = clf.predict(X.iloc[test,:])
        y_pred_prob = clf.predict_proba(X.iloc[test,:])[:,1]
        roc_auc.append(metrics.roc_auc_score(y.iloc[test], y_pred_prob))
        pr_auc.append(metrics.average_precision_score(y.iloc[test], y_pred_prob))
        precision.append(metrics.precision_score(y.iloc[test], y_pred))
        recall.append(metrics.recall_score(y.iloc[test], y_pred))

    print ('ROC-AUC:', np.mean(roc_auc))
    print ('PR-AUC:', np.mean(pr_auc))
    print ('Precission:', np.mean(precision))
    print ('Recall:', np.mean(recall))
    
    return roc_auc, pr_auc, precision, recall, models

In [9]:
from sklearn.model_selection import StratifiedKFold

In [10]:
# разобьем рабочие данные для кросс-валидации, поделим на 9 подвыборок
skf = StratifiedKFold(n_splits=9, random_state=19)

In [11]:
%%time

# Логистическая регрессия
clf = LogisticRegression(n_jobs=-1)
roc_auc, pr_auc, precision, recall, models = get_cv_roc_pr_rc(clf, skf, x_data, y_data)

c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the

ROC-AUC: 0.6190087552268136
PR-AUC: 0.5927146748669794
Precission: 0.5900454112586759
Recall: 0.6179421879761438
Wall time: 48.8 s


c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [12]:
%%time

# Случайный лес
clf = RandomForestClassifier(n_estimators = 100, n_jobs=-1)
roc_auc, pr_auc, precision, recall, models = get_cv_roc_pr_rc(clf, skf, x_data, y_data)

c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was

ROC-AUC: 0.678314831816033
PR-AUC: 0.6599552807660232
Precission: 0.6356458928246257
Recall: 0.6322700361325149
Wall time: 6.23 s


In [13]:
%%time

# Градиентный бустинг
clf = GradientBoostingClassifier(n_estimators = 30, random_state = 19)
roc_auc, pr_auc, precision, recall, models = get_cv_roc_pr_rc(clf, skf, x_data, y_data)

c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\dmitry\appdata\local\programs\python\python37-32\lib\site-packag

ROC-AUC: 0.7162289227819557
PR-AUC: 0.7085688315148213
Precission: 0.6455435633343738
Recall: 0.6789067374805915
Wall time: 8.42 s


Были обучены три модели: логистическая регрессия, случайный лес и градиентный бустинг. Все три модели показали примерно одинаковое качество, за отправные значения примем метрики, показанные Градиентным бустингом:

ROC-AUC: 0.716606066648193
Precission: 0.6460469878765588
Recall: 0.6843121036669424